<a href="https://colab.research.google.com/github/zabady9/ICPC-competition-/blob/main/CNNs_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle_api"   # folder path


In [5]:
%mkdir /content/data
%cd /content/data

/content/data


In [6]:
! kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition


100% 1.98G/1.98G [00:14<00:00, 130MB/s]
100% 1.98G/1.98G [00:15<00:00, 142MB/s]


In [7]:
%%capture --no-display 
! unzip /content/data/fruit-and-vegetable-image-recognition.zip


In [8]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread('/content/data/Data_blance/Train/Normal/IM-0001-0001.jpeg') #open only one photo

cv2_imshow(img)

AttributeError: ignored

In [ ]:
import os 
imges = os.listdir('/content/data/Data_blance/Train/Normal')
imges

In [ ]:
import glob
images_0 = glob.glob('/content/data/Data_blance/Train/Normal/*.jpeg')
images_0


In [ ]:
train_path = '/content/data/train'
test_path = '/content/data/test'
val_path = '/content/data/validation'

In [ ]:
batch_size = 10
image_size = 224

train_dataset = tf.keras.preprocessing.image_dataset_from_directory( 
    train_path,
    shuffle = True,
    image_size=(image_size,image_size),
    batch_size = batch_size
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    shuffle = True,
    image_size=(image_size,image_size),
    batch_size = batch_size
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_path,
    shuffle = True,
    image_size=(image_size,image_size),
    batch_size = batch_size
)


In [ ]:
class_names=train_dataset.class_names
class_names

In [ ]:

plt.figure(figsize=(15,15))
for image_batch , label_batch in train_dataset.take(1):
  for i in range(9):#9=3*3
    ax = plt.subplot(3,3,i+1)
    plt.imshow(image_batch[i].numpy().astype('uint8'))
    plt.title(class_names[label_batch[i]],fontsize=16)
    plt.axis('off')


In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#image_size=200
model = Sequential(
    [
        layers.Conv2D(128,9,padding='same',activation='relu',input_shape=(image_size,image_size,3)),
        layers.MaxPool2D(),
        # layers.Dropout(0.50),
     
        layers.Conv2D(64,6,padding='same',activation='relu',),
        layers.MaxPool2D(),
        # layers.Dropout(0.50),


        layers.Conv2D(32,3,padding='same',activation='relu',),
        layers.MaxPool2D(),
        # layers.Dropout(0.50),


        layers.Conv2D(16,3,padding='same',activation='relu',),
        layers.MaxPool2D(),
        # layers.Dropout(0.50),
        layers.Conv2D(8,1,padding='same',activation='relu',),
        layers.MaxPool2D(),
        # layers.Dropout(0.50),
     
        layers.Flatten(),
     
        layers.Dense(200 , activation='relu'),
        layers.Dense(128 , activation='relu'),
        layers.Dense(64 , activation='relu'),
        layers.Dense(32 , activation='relu'),     
        layers.Dense(16 , activation='relu'),
        layers.Dense(3 , activation='softmax'),
    ]
)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [ ]:
# Epochs = 5

his = model.fit(
    train_dataset,
 epochs = 15,
 batch_size = batch_size,
 validation_data = val_dataset
)

In [ ]:
his.history

In [ ]:
acc = his.history['accuracy']
val_acc = his.history['val_accuracy']
loss = his.history['loss']
val_loss = his.history['val_loss']

In [ ]:
acc

In [ ]:
def predict(model,img):
  img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
  img_array = tf.expand_dims(img_array,0)

  predictions = model.predict(img_array)
  predictions_class = class_names[np.argmax(predictions[0])]
  conf = round(100*(np.max(predictions[0])),2)
  return predictions_class , conf

In [ ]:
plt.figure(figsize=(15,15))
for images , labels in test_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    pred,conf = predict(model,images[i].numpy())
    acu_cl = class_names[labels[i]]

    plt.title(f"Acual : {acu_cl},\n Pred : {pred}\n conf : {conf}",fontsize=16)
    plt.axis('off')
